In [21]:
import cv2
import time
import numpy as np


def threshImageBg():
    # Load the image
    image = cv2.imread("/home/nata-brain/camera_ws/src/EyeHeadTrack/vision/report/heatmap.png")

    # Define the white color range in BGR format
    lower_white = np.array([200, 200, 200], dtype=np.uint8)
    upper_white = np.array([255, 255, 255], dtype=np.uint8)

    # Create a mask of white pixels within the defined range
    mask = cv2.inRange(image, lower_white, upper_white)

    # Invert the mask to select non-white pixels
    mask_inv = cv2.bitwise_not(mask)

    # Apply the mask to remove white pixels
    result = cv2.bitwise_and(image, image, mask=mask_inv)

    # Save the result image
    cv2.imwrite("/home/nata-brain/camera_ws/src/EyeHeadTrack/vision/report/heatmap1.jpg", result)


# ADD Images
def addImages():
    base_image = cv2.imread("/home/nata-brain/camera_ws/src/EyeHeadTrack/vision/test_images/grocery.jpg")
    threshImageBg()
    overlay_image = cv2.imread("/home/nata-brain/camera_ws/src/EyeHeadTrack/vision/report/heatmap1.jpg")
    
    # Resize the overlay image to match the dimensions of the base image
    overlay_image = cv2.resize(overlay_image, (base_image.shape[1], base_image.shape[0]))

    # Convert the images to grayscale
    base_gray = cv2.cvtColor(base_image, cv2.COLOR_BGR2GRAY)
    overlay_gray = cv2.cvtColor(overlay_image, cv2.COLOR_BGR2GRAY)

    # Create a mask by thresholding the overlay image
    _, mask = cv2.threshold(overlay_gray, 15, 255, cv2.THRESH_BINARY)

    # Invert the mask
    mask_inv = cv2.bitwise_not(mask)

    # Use bitwise operations to apply the mask and overlay the images
    base_masked = cv2.bitwise_and(base_image, base_image, mask=mask_inv)
    overlay_masked = cv2.bitwise_and(overlay_image, overlay_image, mask=mask)
    result = cv2.add(base_masked, overlay_masked)

    # Save the resulting image
    cv2.imwrite("/home/nata-brain/camera_ws/src/EyeHeadTrack/vision/report/result_image.jpg", result)
    
if __name__ == '__main__':
    addImages()

In [50]:
#!/home/nata-brain/miniconda3/envs/eyegaze/bin/python
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
sns.set(style="whitegrid",rc={"figure.figsize": (19.2, 10.8)})

# Lib to data processing 
class dbscanAlgo:
    def __init__(self):
        self.df                 = pd.read_csv('/home/nata-brain/camera_ws/src/EyeHeadTrack/vision/dataset/gaze_points.csv') 
        self.X_train            = self.df
        self.clustering         = DBSCAN(eps=12.5, min_samples=4).fit(self.X_train)
        self.DBSCAN_dataset     = self.X_train.copy()
        self.outliers           = ''
        self.img                = np.asarray(Image.open('/home/nata-brain/camera_ws/src/EyeHeadTrack/vision/test_images/grocery.jpg'))
        self.path_save          = '/home/nata-brain/camera_ws/src/EyeHeadTrack/vision/report/'
        self.path_              = '/home/nata-brain/camera_ws/src/EyeHeadTrack/vision/dataset/'
        self.df_plot            = ''
        
    # Processing dataframe    
    def dfHandling(self):
        self.DBSCAN_dataset.loc[:,'Cluster'] = self.clustering.labels_ 
        self.DBSCAN_dataset.Cluster.value_counts().to_frame()
        self.outliers = self.DBSCAN_dataset[self.DBSCAN_dataset['Cluster']==-1]
        self.df = self.DBSCAN_dataset[self.DBSCAN_dataset['Cluster'] !=-1]
        
    def plotClusters(self):    
        plt.figure(figsize=(19.2, 10.8))
        sns.scatterplot(data = self.df, x = "x", y = "y", hue = self.df.Cluster, legend = "full", palette = "deep")
        plt.savefig(f'{self.path_save}clusters.png')        
        plt.xlim(0, 1920)
        plt.ylim(0, 1080)


In [53]:
obj = dbscanAlgo()
obj.df

,Unnamed: 0,timestamp,x,y
0,0,1.689258e+09,1680.000000,55.446429
1,1,1.689258e+09,1470.000000,115.533482
2,2,1.689258e+09,1286.250000,165.698940
3,3,1.689258e+09,1125.468750,203.808001
4,4,1.689258e+09,1002.239702,229.921286
...,...,...,...,...
335,335,1.689258e+09,998.264662,865.519407
336,336,1.689258e+09,893.117943,859.543767
337,337,1.689258e+09,781.478200,854.797224
338,338,1.689258e+09,683.793425,853.054714


In [54]:
obj.dfHandling()
obj.df.timestamp

46     1.689258e+09
47     1.689258e+09
48     1.689258e+09
49     1.689258e+09
50     1.689258e+09
           ...     
319    1.689258e+09
320    1.689258e+09
321    1.689258e+09
322    1.689258e+09
323    1.689258e+09
Name: timestamp, Length: 79, dtype: float64

In [55]:
obj.plotClusters

<bound method dbscanAlgo.plotClusters of <__main__.dbscanAlgo object at 0x7f9b74ac2d60>>